## Group comparison in python 

import libaries

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg

Generate Example Data

In [ ]:
np.random.seed(42)
data = pd.DataFrame({
    'Group': np.random.choice(['A', 'B'], 100),
    'Score': np.random.normal(50, 10, 100),
    'Subject': np.arange(100),
    'Time1': np.random.normal(50, 10, 100),
    'Time2': np.random.normal(52, 10, 100),
    'Covariate1': np.random.normal(100, 15, 100),
    'Covariate2': np.random.normal(100, 15, 100),})

# longitudinal dataframe
data_long = data.melt(id_vars=['Group', 'Subject', 'Score', 'Covariate1', 'Covariate2'], value_vars=['Time1', 'Time2'], 
                      var_name='Time', value_name='Time_Score')

Independent t-test

In [ ]:
ind_ttest = stats.ttest_ind(
    data.loc[data['Group'] == 'A', 'Score'],
    data.loc[data['Group'] == 'B', 'Score'],
    equal_var=True)
print("Independent t-test:", ind_ttest)

Paired t-test

In [ ]:
paired_ttest = stats.ttest_rel(data['Time1'], data['Time2'])
print("Paired t-test:", paired_ttest)

One-way ANOVA

In [ ]:
# statsmodels
anova = smf.ols('Score ~ Group', data=data).fit()
anova_result = sm.stats.anova_lm(anova, typ=2)

# pingouin
anova_result = pg.anova(data=data, dv='Score', between='Group').round(3)

print("One-way ANOVA:", anova_result)

Repeated Measures ANOVA

In [ ]:
rm_anova = pg.rm_anova(data=data_long, dv='Time_Score', within='Time', subject='Subject', detailed=True)
print("Repeated Measures ANOVA:", rm_anova)

Mixed ANOVA

In [ ]:
mixed_anova = pg.mixed_anova(data=data_long, dv='Time_Score', within='Time', between='Group', subject='Subject')
print("Mixed ANOVA:", mixed_anova)

ANCOVA

In [ ]:
# statsmodels
ancova = smf.ols('Score ~ Group + Covariate1 + Covariate2', data=data).fit()
ancova_result = sm.stats.anova_lm(ancova, typ=2)

# pingouin
ancova_result = pg.ancova(data=data, dv='Score', between='Group', covar=['Covariate1', 'Covariate2']).round(3) 

print("ANCOVA:", ancova_result)

Linear Mixed Model (LMM)

In [ ]:
lmm = smf.mixedlm('Score ~ Group + Covariate1 + Covariate2', data, groups=data['Subject']).fit()
print("Linear Mixed Model:", lmm.summary())

# longitudinal data:
lmm_long = smf.mixedlm('Time_Score ~ Time * Group + Covariate1 + Covariate2', data_long, groups=data_long['Subject']).fit()
print("Linear Mixed Model:", lmm_long.summary())

Generalized Linear Mixed Model (GLMM)

In [ ]:
glmm = smf.glm('Score ~ Group + Covariate1 + Covariate2', data, groups=data['Subject'], family=sm.families.Binomial()).fit()
print("Generalized Linear Mixed Model:", glmm.summary())

# longitudinal data:
glmm_long = smf.glm('Time_Score ~ Time * Group + Covariate1 + Covariate2', data_long, groups=data_long['Subject'], family=sm.families.Binomial()).fit()
print("Generalized Linear Mixed Model:", glmm_long.summary())